# Status #

- [x] Outline
- [x] Introduction
- [ ] Exercise 1
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Exercise 2
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Exercise 3
  - [x] Code
  - [x] Discussion
  - [ ] Checking
- [ ] Exercise 4
  - [ ] Code
  - [x] Discussion
  - [ ] Checking
- [x] Conclusion

# Introduction #

In these exercises, you'll explore the effect of the `strides` and `padding` parameters in `Conv2D` and `MaxPool2D` layers, learn about how convnets can capture large-scale visual features through stacking layers, and finally see how convolution can be used on one-dimensional data, a time series.

Run the cell below to set everything up.

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.computer_vision.ex4 import *

# Moving Windows #

### 1) Explore Window Sizes

In [ ]:

q_1.check()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_1.hint()
#_COMMENT_IF(PROD)_
q_1.solution()

### 2) Explore Stride


In [ ]:

q_2.check()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_2.hint()
#_COMMENT_IF(PROD)_
q_2.solution()

# The Receptive Field #

In all of the examples we've done, we've used $3 \times 3$ kernels; this is perhaps the most common choice for kernel size in convolutional networks. We might worry, though: if our images have dimension `(192, 192)`, are kernels with dimension `(3, 3)` large enough to capture all of the important features? A $3 \times 3$ square wouldn't cover the shape of an eye or an ear, for instance. Perhaps we should use larger kernels?

In fact, this is rarely necessary. Networks occassionally will have an initial `Conv2D` layer with larger kernels, perhaps with `kernel_size=(5, 5)`, but usually not much larger. Instead, convnets can more effectively capture large-scale information from an image by stacking convolution and pooling layers. This stacking increases the number of pixels the output neurons are receiving information from, that is, it increases the size of the neurons' **receptive field**. Let's see how this happens now.

### 3) How the Receptive Field Grows

This next picture illustrates two stacked convolutional layers with `(3, 3)` kernels. Notice how the total number of neurons connected to the output neuron at top has increased.

Look at the dimensions of the square of the connected neurons at bottom. What effect did stacking two `Conv2D` layers with `kernel_size=(3, 3)` have on the effective window size for the neurons in the output layer?

In [ ]:
# Lines below will give you a hint or solution
#_COMMENT_IF(PROD)_
q_3.a.hint()
#_COMMENT_IF(PROD)_
q_3.a.solution()

What would happen if you added a `(2, 2)` maximum pooling layer with `strides=2` after the convolutions?

In [ ]:
# Lines below will give you a hint or solution
#_COMMENT_IF(PROD)_
q_3.b.hint()
#_COMMENT_IF(PROD)_
q_3.b.solution()

In the next lesson, you'll learn how to create convolutional networks like this:

```
model = keras.Sequential([
    # Base
    layers.Conv2D(filters=64, kernel_size=3, strides=1, activation='relu'),
    layers.Conv2D(filters=64, kernel_size=3, strides=1, activation='relu'),
    layers.MaxPool2D(pool_size=2, strides=2),
    layers.Conv2D(filters=128, kernel_size=3, strides=1, activation='relu'),
    layers.MaxPool2D(pool_size=2, strides=2),
    # Head follows
    # ...
])
```

What is the receptive field of a neuron following the second `MaxPool2D` layer? (This one's a bit tricky. You can run the `hint` several times to get more hints, if you need.)

In [ ]:
#_COMMENT_IF(PROD)_
q_3.c.hint()
#_COMMENT_IF(PROD)_
q_3.c.solution()

# One-Dimensional Convolution #

Though we've been using convolutional networks on two-dimensional data, it turns out that they can also be useful on *one*-dimensional data, like time series or natural language texts. In fact, convolutional networks tend to be successful on any kind of data with a strong **local topological structure**, meaning that the information about a point tends to be concentrated in nearby points -- you can most successfully predict the value of a pixel by looking at nearby pixels, you can most successfully predict the weather today by looking at the weather yesterday instead of a month ago.

### 4) Apply a 1D Convolution

In this exercise, we'll see how a convolution can be used on a **time series**. The time series we'll use is from [Google Trends](https://trends.google.com/trends/); it measures the popularity of the search term "machine learning" for weeks from January 25, 2015 to January 15, 2020.

In [ ]:
import pandas as pd

# Load the time series as a Pandas dataframe
machinelearning = pd.read_csv('../data/machinelearning.csv',
                              parse_dates=['Week'],
                              index_col='Week')

machinelearning.plot();

Because our data is one-dimensional, the kernel needs to be one-dimensional as well. Define a one dimensional kernel. Though not required, you'll get better results if the entries sum to 1.

In [ ]:
# YOUR CODE HERE: Define a 1D kernel.
kernel = tf.constant([____])
q_4.check()

In [ ]:
#%%RM_IF(PROD)%%
kernel = tf.constant([0.2, 0.6, 0.2])
q_4.assert_check_passed()

Now run the next cell to apply the kernel with a convolution and see what effect it had on the time series.

In [ ]:
# Reformat for TensorFlow
ts_data = machinelearning.to_numpy()[:, 1]
ts_data = tf.expand_dims(ts_data, axis=0)

ts_filter = tf.nn.conv2d(
    input=ts_data,
    filters=kernel,
    strides=1,
    padding='VALID',
)

# Format as Pandas Series
machinelearning_filtered = pd.Series(tf.squeeze(ts_filter).numpy())

machinelearning_filtered.plot()

# Conclusion #

This lesson ends our discussion of feature extraction. Hopefully, having completed these lessons, you've gained some intuition about how the process works and why the usual choices for its implementation are often the best ones.

In the next lesson, Lesson 5, you'll learn how to compose the `Conv2D` and `MaxPool2D` layers to build your own convolutional networks from scratch.